In [0]:
#!pip install tensorflow-gpu

In [0]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd
from scipy.stats import zscore
#from clr_callback import CyclicLR

In [3]:
print(tf.__version__)
tf.executing_eagerly()

2.0.0


True

In [4]:
for f in os.listdir('.'):
  print(f)

.config
test.csv
__pycache__
train.csv
predictions.csv
clr_callback.py
sample_data


In [0]:
train = pd.read_csv('train.csv')

In [0]:
train = train.sample(frac=1).reset_index(drop=True)

In [7]:
train.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,98,1,1,"Greenfield, Mr. William Bertram",male,23.0,0,1,PC 17759,63.3583,D10 D12,C
1,14,0,3,"Andersson, Mr. Anders Johan",male,39.0,1,5,347082,31.2750,NaN,S
2,877,0,3,"Gustafsson, Mr. Alfred Ossian",male,20.0,0,0,7534,9.8458,NaN,S
3,633,1,1,"Stahelin-Maeglin, Dr. Max",male,32.0,0,0,13214,30.5000,B50,C
4,532,0,3,"Toufik, Mr. Nakli",male,NaN,0,0,2641,7.2292,NaN,C
5,250,0,2,"Carter, Rev. Ernest Courtenay",male,54.0,1,0,244252,26.0000,NaN,S
6,40,1,3,"Nicola-Yarred, Miss. Jamila",female,14.0,1,0,2651,11.2417,NaN,C
7,433,1,2,"Louch, Mrs. Charles Alexander (Alice Adelaide ...",female,42.0,1,0,SC/AH 3085,26.0000,NaN,S
8,881,1,2,"Shelley, Mrs. William (Imanita Parrish Hall)",female,25.0,0,1,230433,26.0000,NaN,S
9,380,0,3,"Gustafsson, Mr. Karl Gideon",male,19.0,0,0,347069,7.7750,NaN,S


In [8]:
print(train['Fare'].max())

512.3292


In [0]:
features = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]
x_train = train[features]

In [10]:
x_train.head(10)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,male,23.0,0,1,63.3583,C
1,3,male,39.0,1,5,31.2750,S
2,3,male,20.0,0,0,9.8458,S
3,1,male,32.0,0,0,30.5000,C
4,3,male,NaN,0,0,7.2292,C
5,2,male,54.0,1,0,26.0000,S
6,3,female,14.0,1,0,11.2417,C
7,2,female,42.0,1,0,26.0000,S
8,2,female,25.0,0,1,26.0000,S
9,3,male,19.0,0,0,7.7750,S


In [0]:
x_train = x_train.replace('male', 0)
x_train = x_train.replace('female', 1)
x_train = x_train.replace('S', 0)
x_train = x_train.replace('C', 1)
x_train = x_train.replace('Q', 2)

In [0]:
x_train = x_train.fillna(0)

In [13]:
x_train.head(10)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,23.0,0,1,63.3583,1.0
1,3,0,39.0,1,5,31.2750,0.0
2,3,0,20.0,0,0,9.8458,0.0
3,1,0,32.0,0,0,30.5000,1.0
4,3,0,0.0,0,0,7.2292,1.0
5,2,0,54.0,1,0,26.0000,0.0
6,3,1,14.0,1,0,11.2417,1.0
7,2,1,42.0,1,0,26.0000,0.0
8,2,1,25.0,0,1,26.0000,0.0
9,3,0,19.0,0,0,7.7750,0.0


In [14]:
mu = []
sigma = []
for i in range(7):
  mu.append(x_train.iloc[0:800, i].mean())
  sigma.append(x_train.iloc[0:800, i].std())
print(mu)
print(sigma)

[2.305, 0.3625, 23.6502125, 0.5425, 0.40125, 32.77906175, 0.36875]
[0.8323707415053381, 0.4810228463632513, 17.718930341456904, 1.132385367609439, 0.823769799977032, 51.19718942162671, 0.6409266113615789]


In [0]:
x_train = pd.DataFrame((x_train.to_numpy() - np.array(mu))/np.array(sigma))

In [16]:
x_train.head(10)

,0,1,2,3,4,5,6
0,-1.567811,-0.753602,-0.036696,-0.479077,0.726841,0.597284,0.984902
1,0.834964,-0.753602,0.866293,0.404014,5.582567,-0.029378,-0.575339
2,0.834964,-0.753602,-0.206006,-0.479077,-0.487090,-0.447940,-0.575339
3,-1.567811,-0.753602,0.471235,-0.479077,-0.487090,-0.044515,0.984902
4,0.834964,-0.753602,-1.334743,-0.479077,-0.487090,-0.499048,0.984902
5,-0.366423,-0.753602,1.712845,0.404014,-0.487090,-0.132411,-0.575339
6,0.834964,1.325301,-0.544627,0.404014,-0.487090,-0.420675,0.984902
7,-0.366423,1.325301,1.035604,0.404014,-0.487090,-0.132411,-0.575339
8,-0.366423,1.325301,0.076178,-0.479077,0.726841,-0.132411,-0.575339
9,0.834964,-0.753602,-0.262443,-0.479077,-0.487090,-0.488387,-0.575339


In [0]:
y_train = np.array(train['Survived'])
x_train = np.array(x_train)

In [0]:
x_cv = x_train[800:, :]
y_cv = y_train[800:]

x_train = x_train[:800, :]
y_train = y_train[:800]

In [0]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(7, activation='relu'),
  tf.keras.layers.Dense(7, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [0]:
inputs = tf.keras.Input(shape=(7))

s = tf.keras.layers.Dropout(0.5)(inputs)
s = tf.keras.layers.Dense(7, activation='relu')(s)
a = tf.keras.layers.Concatenate()(list([s, inputs]))

s = tf.keras.layers.Dropout(0.5)(a)
s = tf.keras.layers.Dense(7, activation='relu')(s)
a = tf.keras.layers.Concatenate()(list([s, a]))

a = tf.keras.layers.Dropout(0.5)(a)
a = tf.keras.layers.Dense(1, activation='sigmoid')(a)

model = tf.keras.Model(inputs=inputs, outputs=a)

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [0]:
inputs = tf.keras.Input(shape=(7))

a = tf.keras.layers.Reshape([7])(inputs)

for i  in range(5):
  s = tf.keras.layers.BatchNormalization()(a)
  s = tf.keras.layers.Dropout(0.5)(s)
  s = tf.keras.layers.Dense(7, activation='relu')(s)
  a = tf.keras.layers.Concatenate()(list([a, s]))
  
a = tf.keras.layers.BatchNormalization()(a)
a = tf.keras.layers.Dropout(0.5)(a)
a = tf.keras.layers.Dense(1, activation='sigmoid')(a)

model = tf.keras.Model(inputs=inputs, outputs=a)

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [22]:


#clr = CyclicLR(base_lr=0.001, max_lr=0.006, step_size=2000.)
#model.fit(x_train, y_train, epochs=1, callbacks=[clr])

model.fit(x_train, y_train, epochs=1000)

model.evaluate(x_train,  y_train, verbose=2)
model.evaluate(x_cv,  y_cv, verbose=2)

Train on 800 samples
Epoch 1/1000
800/800 [==============================] - 3s 3ms/sample - loss: 0.8545 - accuracy: 0.5738
Epoch 2/1000
800/800 [==============================] - 0s 226us/sample - loss: 0.7990 - accuracy: 0.5987
Epoch 3/1000
800/800 [==============================] - 0s 226us/sample - loss: 0.7817 - accuracy: 0.6175
Epoch 4/1000
800/800 [==============================] - 0s 214us/sample - loss: 0.6738 - accuracy: 0.6562
Epoch 5/1000
800/800 [==============================] - 0s 234us/sample - loss: 0.6775 - accuracy: 0.6562
Epoch 6/1000
800/800 [==============================] - 0s 231us/sample - loss: 0.7133 - accuracy: 0.6812
Epoch 7/1000
800/800 [==============================] - 0s 227us/sample - loss: 0.6010 - accuracy: 0.7113
Epoch 8/1000
800/800 [==============================] - 0s 228us/sample - loss: 0.6304 - accuracy: 0.7063
Epoch 9/1000
800/800 [==============================] - 0s 218us/sample - loss: 0.6109 - accuracy: 0.7063
Epoch 10/1000
800/800 [====

[0.4619451545097016, 0.8131868]

In [0]:
test = pd.read_csv('test.csv')
x_test = test[features]
x_test = x_test.replace('male', 0)
x_test = x_test.replace('female', 1)
x_test = x_test.replace('S', 0)
x_test = x_test.replace('C', 1)
x_test = x_test.replace('Q', 2)
x_test = x_test.fillna(0)
x_test = pd.DataFrame((x_test.to_numpy() - np.array(mu))/np.array(sigma))
x_test = np.array(x_test)

In [0]:
predictions = model.predict(x_test)

In [25]:
type(predictions)

numpy.ndarray

In [26]:
ids = test['PassengerId'].to_numpy().reshape([-1, 1])

print(ids.shape, predictions.shape)

predictions = np.concatenate([ids, predictions], -1)

(418, 1) (418, 1)


In [27]:
print(predictions, predictions.shape)

[[8.92000000e+02 1.25776112e-01]
 [8.93000000e+02 3.89945477e-01]
 [8.94000000e+02 1.24632239e-01]
 [8.95000000e+02 1.23729169e-01]
 [8.96000000e+02 4.68200028e-01]
 [8.97000000e+02 1.31624550e-01]
 [8.98000000e+02 7.57271051e-01]
 [8.99000000e+02 3.64243329e-01]
 [9.00000000e+02 7.17486322e-01]
 [9.01000000e+02 1.70333475e-01]
 [9.02000000e+02 1.32672369e-01]
 [9.03000000e+02 2.66678393e-01]
 [9.04000000e+02 9.50336814e-01]
 [9.05000000e+02 1.54970288e-01]
 [9.06000000e+02 9.48582590e-01]
 [9.07000000e+02 9.13010538e-01]
 [9.08000000e+02 1.74536288e-01]
 [9.09000000e+02 1.33381873e-01]
 [9.10000000e+02 4.68590409e-01]
 [9.11000000e+02 6.39945269e-01]
 [9.12000000e+02 2.42979676e-01]
 [9.13000000e+02 3.27551126e-01]
 [9.14000000e+02 9.45288181e-01]
 [9.15000000e+02 4.49987888e-01]
 [9.16000000e+02 9.49444711e-01]
 [9.17000000e+02 1.18761331e-01]
 [9.18000000e+02 9.49248016e-01]
 [9.19000000e+02 1.32585764e-01]
 [9.20000000e+02 2.87811667e-01]
 [9.21000000e+02 1.99489027e-01]
 [9.220000

In [0]:
predictions = pd.DataFrame(predictions)

In [0]:
predictions.columns = ['PassengerId', 'Survived']

In [0]:
predictions = predictions.round().astype('int32')

In [0]:
predictions.to_csv('predictions.csv', index=False)

In [32]:
predictions.head(10)

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
5,897,0
6,898,1
7,899,0
8,900,1
9,901,0
